# Livelier Libraries

#### Table of contents \
#### 1. Acquire some lists of libraries in the UK \
#### 2. Acquire some comments of the libraries \
#### 3. Acquire the data of users

##### Import relevant libraries

In [1]:
import requests 
import pandas as pd 
import numpy as np 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.4                |           py36_0         877 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         961 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The following packages will be UPDATED:

  conda                                        4.6.3-py36_0 --> 4.6.4-py36_0
  geopy              conda-forge/linux-64::geopy-1.11.0-py~ --> conda-forge/noarch::geopy-1.18.1-py_0



geopy-1.18.1         

In [1]:
# Define my Foursquare API credentials
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605'


### 1. Acquires some lists of libraries in the UK

#### Set my starting point at London and get the geographic coordinates

In [5]:
geolocator = Nominatim(user_agent="london")
location = geolocator.geocode("Holborn station, London")
print((location.latitude, location.longitude))

(51.5176863, -0.1185622)


#### Foursquare API query to look up venues in the Library catefory

In [7]:
LIMIT = 100 
radius = 5000
categoryId ='4bf58dd8d48988d12f941735' #Foursquare Category ID for Libraries

url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    categoryId,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    location.latitude, 
    location.longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d12f941735&client_id=H5U3C5TGF5I4OM0C2WBGW0FCWIIODSZMJXIPONYVUJYISWGL&client_secret=GLHA2ZYV5CLZJJ4ZNWI04FLHT1YVJYMZ53NKQJNL35IOC112&v=20180605&ll=51.5176863,-0.1185622&radius=5000&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c67f2e81ed2196e46ee9343'},
 'response': {'venues': [{'id': '4ceea4c0f368b1f761c481f7',
    'name': 'City Law School Library',
    'location': {'address': "Gray's Inn",
     'lat': 51.519017898682854,
     'lng': -0.11443017936033537,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.519017898682854,
       'lng': -0.11443017936033537}],
     'distance': 322,
     'cc': 'GB',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ["Gray's Inn",
      'London',
      'Greater London',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d12f941735',
      'name': 'Library',
      'pluralName': 'Libraries',
      'shortName': 'Library',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1550316265',
    'hasPerk': False},
   {'id': '4ac518cef964a52019a620e3',
    

#### Transform the relevant part of the result into a dataframe

In [10]:
venues = results['response']['venues']

libraries_lnd = json_normalize(venues)

#Clean the dataframe
lnd=libraries_lnd[['name','location.lat','location.lng','id']]
lnd.head()

,name,location.lat,location.lng,id
0,City Law School Library,51.519018,-0.114430,4ceea4c0f368b1f761c481f7
1,British Library,51.529610,-0.127490,4ac518cef964a52019a620e3
2,Explore history,51.495939,-0.108531,5948e18bf62e090a3ba49e23
3,West Library,51.540778,-0.116081,4f5b6735e4b08380674c2a07
4,Barbican Children's Library,51.519502,-0.093278,4c57f4ae744a0f47690bcacb


#### Define a function to get Library list, when inputs a landmark in a city

In [14]:
def getLibrary (Landmark, City):
    
    geolocator = Nominatim(user_agent="place")
    location = geolocator.geocode(Landmark, City)
    location.latitude, location.longitude
    
    LIMIT = 100 
    radius = 5000
    categoryId ='4bf58dd8d48988d12f941735'
    
    url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categoryId,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        location.latitude, 
        location.longitude,
        radius,
        LIMIT)
    
    results = requests.get(url).json()
    venues = results['response']['venues']
    libraries = json_normalize(venues)
    libraries=libraries[['name','location.lat','location.lng','id']]
    
    return(libraries)

#### Get the libraries for the 5 major cities in UK

In [21]:
manchester=getLibrary(Landmark="Manchester town hall", City="Manchester")
manchester.shape

(27, 4)

In [19]:
birmingham=getLibrary(Landmark="Cathedral Square", City="Birmingham")
birmingham.shape

(50, 4)

In [20]:
leeds=getLibrary(Landmark="Leeds Station", City="Leeds")
leeds.shape

(31, 4)

In [22]:
glasgow=getLibrary(Landmark="Glasgow Central Station", City="Glasgow")
glasgow.shape

(31, 4)

In [24]:
liverpool=getLibrary(Landmark="Liverpool Central station", City="liverpool")
liverpool.shape

(24, 4)

In [28]:
#Combines the library dataframes
uk_libraries=pd.concat([lnd,manchester,birmingham,leeds,glasgow,liverpool], ignore_index=True)
print(uk_libraries.shape)
uk_libraries.tail()

(213, 4)


,name,location.lat,location.lng,id
208,Fade Library Delivery Service,53.414356,-2.922605,4f3a9ccde4b07324287bfef2
209,Wavertree Library,53.401168,-2.923971,4be3066df07b0f47c1e0f643
210,Childwall Library,53.397831,-2.897607,515455c7e4b00958a04fc70b
211,St James Centre,53.401712,-3.057477,5350073f498eef46f38d7c0b
212,Fade Evidence Knowledge Centre,53.404235,-2.923725,4d93158d7b5ea1436d51bfb8


### 2. Acquire the comments for the Libraries

#### First creates empty dataframe to fill with comments later

In [26]:
column_names = ['Library_id', 'Comment', 'Likes', 'Dislikes', 'User_id'] 

library_reviews = pd.DataFrame(columns=column_names)
library_reviews

,Library_id,Comment,Likes,Dislikes,User_id


#### Loop the comments into the dataframe

In [30]:
for index, row in uk_libraries.iterrows():
    row['id']
    
    url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit=1'.format(
        row['id'], CLIENT_ID, CLIENT_SECRET, VERSION)
    reviews = requests.get(url).json()['response']['tips']['items']
    
    for data in reviews:
        ID=data['id']
        comment=data['text']
        likes=data['agreeCount']
        dislikes=data['disagreeCount']
        user_id=data['user']['id']
        
        library_reviews=library_reviews.append({'Library_id':ID,
                                               'Comment':comment,
                                               'Likes':likes,
                                               'Dislikes':dislikes,
                                               'User_Id':user_id}, ignore_index=True)


In [31]:
print(library_reviews.shape)
library_reviews.head()

(93, 6)


,Library_id,Comment,Likes,Dislikes,User_id,User_Id
0,59bd8da2e4c4596abf8e9fa0,"Free admission. See the Magna Carta, original ...",11,0,NaN,1352733
1,55b1d6c9498ed7b07ac19aef,Well utilized space. Especially liked the musi...,0,0,NaN,38979216
2,5145bed8e4b09257139b4ec8,Cafe on top floor with lots of newspapers and ...,1,0,NaN,1769757
3,55a23750498e3d428e995705,"A great, quiet space to sit and read the Wellc...",0,0,NaN,1940876
4,51374364e4b00eec86871dc9,checkout the notice boards for skill swaps & w...,1,0,NaN,10069948


In [32]:
# Save the data into CSV
library_reviews.to_csv("uk_library_reviews.csv", sep=',')